<a href="https://colab.research.google.com/github/Yashu2699/Deep_learning/blob/main/Semantic_seg/sandstone_dataset_multiclass_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=621bc08aac7b04cd2bd16f534fb06ac5c65f12e4efbea369837b2f7711785c4f
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [2]:
%cd '/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation'

/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation


In [3]:
import import_ipynb
import UNET_MultiClass
from UNET_MultiClass import multi_unet

importing Jupyter notebook from UNET_MultiClass.ipynb


In [4]:
!pip install patchify

In [5]:
from keras.utils.np_utils import normalize
import os
import glob
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify
from pathlib import Path

In [6]:
model = multi_unet(4, 128, 128, 1)
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 128, 128, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [7]:
model.load_weights('/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation/h5_files/sandstone_multi_seg.hdf5')

In [12]:
segm_images = []
for directory_path in sorted(glob.glob("/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation/large_dataset")):
  for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
    img = cv2.imread(img_path, 0)
    print(type(img))
    patches = patchify(img, (128, 128), step=128) 
    predicted_patches = []
    for i in range(patches.shape[0]):
      for j in range(patches.shape[1]): 
        single_patch = patches[i,j,:,:]           
        single_patch_norm = np.expand_dims(normalize(np.array(single_patch), axis=1),2)
        single_patch_input=np.expand_dims(single_patch_norm, 0)
        single_patch_prediction = (model.predict(single_patch_input))
        single_patch_predicted_img=np.argmax(single_patch_prediction, axis=3)[0,:,:]   
        predicted_patches.append(single_patch_predicted_img)
    
    predicted_patches = np.array(predicted_patches)
    
    predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 128,128) )
    
    reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)

    segm_images.append(reconstructed_image)
    print("Finished segmenting image: ", name)


<class 'NoneType'>


TypeError: ignored

In [10]:
segm_images = []
path = "/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation/sandstone_data/*.jpg"
from pathlib import Path
for file in glob.glob(path):
    #print(file)     #just stop here to see all file names printed
    name = Path(file).stem #Get the original file name
    #print(name)
  
    large_image = cv2.imread(file, 0)
    large_image = cv2.resize(large_image, (1024, 896))
    patches = patchify(large_image, (128, 128), step=128)  #Step=256 for 256 patches means no overlap
    
    predicted_patches = []
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            #print(i,j)
            
            single_patch = patches[i,j,:,:]
            
            single_patch_norm = np.expand_dims(normalize(np.array(single_patch), axis=1),2)
            single_patch_input=np.expand_dims(single_patch_norm, 0)
    
            single_patch_prediction = (model.predict(single_patch_input))
            single_patch_predicted_img=np.argmax(single_patch_prediction, axis=3)[0,:,:]
    
            predicted_patches.append(single_patch_predicted_img)
    
    predicted_patches = np.array(predicted_patches)
    
    predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 128,128) )
    
    reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
    #Here you can save individual images, or save on milti_dim tiff file
    #cv2.imwrite('segmented_images/' + name + '_segmented.tif', reconstructed_image)
    segm_images.append(reconstructed_image)
    print("Finished segmenting image: ", name)

Finished segmenting image:  sandstone_all_462_images-1
Finished segmenting image:  sandstone_all_462_images-2
Finished segmenting image:  sandstone_all_462_images-3
Finished segmenting image:  sandstone_all_462_images-4
Finished segmenting image:  sandstone_all_462_images-5
Finished segmenting image:  sandstone_all_462_images-6
Finished segmenting image:  sandstone_all_462_images-7
Finished segmenting image:  sandstone_all_462_images-8
Finished segmenting image:  sandstone_all_462_images-9
Finished segmenting image:  sandstone_all_462_images-10
Finished segmenting image:  sandstone_all_462_images-11
Finished segmenting image:  sandstone_all_462_images-12
Finished segmenting image:  sandstone_all_462_images-13
Finished segmenting image:  sandstone_all_462_images-14
Finished segmenting image:  sandstone_all_462_images-15
Finished segmenting image:  sandstone_all_462_images-16
Finished segmenting image:  sandstone_all_462_images-17
Finished segmenting image:  sandstone_all_462_images-18
F

In [11]:
final_segm_image = np.array(segm_images).astype(np.uint8)   

from tifffile import imsave
imsave('/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation/sandstone_segmented_test.tif', final_segm_image)

In [12]:
np.unique(final_segm_image)

array([0, 1, 2, 3], dtype=uint8)